In [ ]:
!pip install ace-tools-open
!pip install git+https://github.com/khanh96le/hl7tojson.git

  Cloning https://github.com/khanh96le/hl7tojson.git to /tmp/pip-req-build-pyvbu205
  Running command git clone --filter=blob:none --quiet https://github.com/khanh96le/hl7tojson.git /tmp/pip-req-build-pyvbu205
  Resolved https://github.com/khanh96le/hl7tojson.git to commit c7563d939f43e8038ad6b0e842e8dd0389e032d9
  Preparing metadata (setup.py) ... done


In [ ]:
from hl7tojson import parser
import json
import ace_tools_open as tools
import pandas as pd
import re
import unicodedata
import spacy

In [ ]:
file_path = 'ORU.hl7'

In [ ]:
with open(file_path, 'r', encoding='utf-8') as f:
    raw = f.read().replace('\r', '\n')

lines = [line for line in raw.split('\n') if line.strip()]

narratives = []
for line in lines:
    if line.startswith('OBX|'):
        fields = line.split('|')
        if len(fields) > 5:
            text = fields[5].strip()
            if text:
                narratives.append(text)

for idx, text in enumerate(narratives, start=1):
    print(f"{idx}. {text}")

1. TOMOGRAFÍA COMPUTADA DE TÓRAX
2. Técnica
3. 499.8 DLP (mGy - cm)
4. Medio de contraste  Iopamidol x 50ml: 80 ml
5. Con escanógrafo multidetector, se realizó una adquisición volumétrica desde los ápices pulmonares hasta los hemidiafragmas, posterior a la administración de medio de contraste endovenoso (Medio de contraste  Iopamidol x 50ml: 80 ml).
6. Indicación clínica: Choque séptico, antecedente linfoma, estudio de foco infeccioso y estadificación.
7. Estudios comparados: Ninguno disponible.
8. Hallazgos
9. Parénquimas Pulmonares: Consolidaciones con broncograma aéreo que compromete los segmentos anterior y posterior del lóbulo superior derecho y los segmentos basal apical del lóbulo inferior derecho. Atelectasias en los segmentos basales posteriores de ambos lóbulos inferiores. Granuloma calcificado de 6 mm en el segmento basal posterior del lóbulo inferior izquierdo. Atelectasias subsegmentarias en los segmentos basales de ambos lóbulos inferiores.
10. Pleura: Derrame pleural de 

In [ ]:
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# Términos a excluir
removal_terms = {"tecnica", "hallazgos", "opinion"}
stopwords = nlp.Defaults.stop_words

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
def normalize_text(text: str) -> str:
    s = text.lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("utf-8")
    s = re.sub(r"[.,:;\-\(\)\[\]\"]+", " ", s)
    s = re.sub(r"[¿\?¡!…]+", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def lemmatize_to_text(text: str) -> str:
    norm = normalize_text(text)
    doc = nlp(norm)
    lemmas = [
        tok.lemma_.lower()
        for tok in doc
        if tok.is_alpha
           and tok.lemma_.lower() not in stopwords
           and tok.lemma_.lower() not in removal_terms
    ]
    return " ".join(lemmas)

In [ ]:
def extract_and_clean_hl7(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as f:
        raw = f.read().replace('\r', '\n')
    narratives = [
        parts[5].strip()
        for line in raw.split('\n')
        if line.startswith('OBX|')
        for parts in [line.split('|')]
        if len(parts) > 5 and parts[5].strip()
    ]
    joined = " ".join(narratives)
    return lemmatize_to_text(joined)

In [ ]:
texto_limpio = extract_and_clean_hl7(file_path)

In [ ]:
print(texto_limpio)

tomografia computado torax tecnico dlp mgy cm contraste iopamidol x ml escanografo multidetector realacer adquisicion volumetrico apiz pulmonar hemidiafragma posterior administracion contraste endovenoso contraste iopamidol x ml indicacion clinico choque septico antecedente linfomo estudio foco infeccioso estadificacion estudio comparado disponible hallazgo parenquima pulmonar consolidación broncograma aereo comprometer segmento posterior lobulo superior derecho segmento basal apical lobulo inferior derecho atelectasia segmento basal posterior lobulo inferior granuloma calcificado mm segmento basal posterior lobulo inferior izquierdo atelectasia subsegmentaria segmento basal lobulo inferior pleura derrame pleural distribucion libre bilateral escaso predominio derecho existir camara neumotorax arbol traqueobronquial tubo endotraqueal extremo distal cm carín curso calibre patron ramificacion normal lesión endoluminal visible metodo corazon pericardio derrame pericardico alcanzar espesor 